In [445]:
import pandas as pd

In [446]:

columns = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base",
    "length", "width", "height", "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower",
    "peak-rpm", "city-mpg", "highway-mpg", "price"]

data = pd.read_csv("../data/imports-85.data", names=columns, na_values="?")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  164 non-null    float64
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       203 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

Missing data found! removing rows with missing data!

In [447]:
dataDropped = data.dropna().reset_index()
dataDropped.info()

print(dataDropped)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              159 non-null    int64  
 1   symboling          159 non-null    int64  
 2   normalized-losses  159 non-null    float64
 3   make               159 non-null    object 
 4   fuel-type          159 non-null    object 
 5   aspiration         159 non-null    object 
 6   num-of-doors       159 non-null    object 
 7   body-style         159 non-null    object 
 8   drive-wheels       159 non-null    object 
 9   engine-location    159 non-null    object 
 10  wheel-base         159 non-null    float64
 11  length             159 non-null    float64
 12  width              159 non-null    float64
 13  height             159 non-null    float64
 14  curb-weight        159 non-null    int64  
 15  engine-type        159 non-null    object 
 16  num-of-cylinders   159 non

For whatever reason, some numbers were written as words! converting to int...

In [448]:
%pip install word2number
from word2number import w2n

strangeColumns = ["num-of-doors", "num-of-cylinders"]
standardColumns = list(set(columns) - set(strangeColumns))

strangeData = dataDropped[strangeColumns]
standardData = dataDropped[standardColumns]

strangeData = strangeData.map(lambda a : w2n.word_to_num(str(a)))

data = pd.concat([standardData, strangeData], axis=1)
data.info()
print(data)

Note: you may need to restart the kernel to use updated packages.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   horsepower         159 non-null    float64
 1   stroke             159 non-null    float64
 2   engine-location    159 non-null    object 
 3   width              159 non-null    float64
 4   body-style         159 non-null    object 
 5   drive-wheels       159 non-null    object 
 6   height             159 non-null    float64
 7   city-mpg           159 non-null    int64  
 8   symboling          159 non-null    int64  
 9   engine-type        159 non-null    object 
 10  fuel-type          159 non-null    object 
 11  curb-weight        159 non-null    int64  
 12  normalized-losses  159 non-null    float64
 13  length             159 non-null    float64
 14  make               159 non-null    object 
 15  fuel-sys


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Strings found! Using one-hot encoding to change data to numeric

In [449]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

strangeColumns = ["engine-location", "body-style", "drive-wheels", "engine-type", "fuel-type", "make", "fuel-system", "aspiration"]
standardColumns = list(set(columns) - set(strangeColumns))

strangeData = pd.DataFrame()
standardData = data[standardColumns]

for column in strangeColumns:
    local_cat = data[[column]]  
    local_cat.sample(10)

    encoder = OneHotEncoder()
    local_cat = encoder.fit_transform(local_cat).toarray()
    local_data = pd.DataFrame(local_cat, columns=list(encoder.categories_[0]))
    strangeData = pd.concat([strangeData, local_data], axis=1)
    print(strangeData)
dataCleaned = pd.concat([strangeData, standardData], axis=1)
print("#" * 100)
print(data)
#data.info()

     front
0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
..     ...
154    1.0
155    1.0
156    1.0
157    1.0
158    1.0

[159 rows x 1 columns]
     front  convertible  hardtop  hatchback  sedan  wagon
0      1.0          0.0      0.0        0.0    1.0    0.0
1      1.0          0.0      0.0        0.0    1.0    0.0
2      1.0          0.0      0.0        0.0    1.0    0.0
3      1.0          0.0      0.0        0.0    1.0    0.0
4      1.0          0.0      0.0        0.0    1.0    0.0
..     ...          ...      ...        ...    ...    ...
154    1.0          0.0      0.0        0.0    1.0    0.0
155    1.0          0.0      0.0        0.0    1.0    0.0
156    1.0          0.0      0.0        0.0    1.0    0.0
157    1.0          0.0      0.0        0.0    1.0    0.0
158    1.0          0.0      0.0        0.0    1.0    0.0

[159 rows x 6 columns]
     front  convertible  hardtop  hatchback  sedan  wagon  4wd  fwd  rwd
0      1.0          0.0      0.0        0.0    1.0 

Unscaled values found! Using Standard scaler to scale

In [450]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

newColumns = list(dataCleaned.columns.values)

target = ["price", "peak-rpm", "horsepower"]
X = dataCleaned[list(set(newColumns) - set(target))]
y = dataCleaned[target]

X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.4, random_state=None)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=None)

train_set_scaled = scaler.fit_transform(X_train)
X_train_scaled_df = pd.DataFrame(train_set_scaled, columns=X_train.columns)
X_train_scaled_df.head()

,idi,stroke,toyota,peugot,ohcv,height,2bbl,4wd,ohc,diesel,...,hardtop,dodge,nissan,fwd,jaguar,mazda,audi,dohc,turbo,bmw
0,-0.282038,-0.275435,-0.483494,-0.235702,-0.235702,1.552330,-0.781133,-0.180579,0.532714,-0.282038,...,-0.180579,-0.180579,-0.380235,-1.436842,0.0,-0.235702,-0.180579,-0.209657,2.405351,-0.180579
1,-0.282038,0.879448,2.068279,-0.235702,-0.235702,-0.795940,-0.781133,-0.180579,0.532714,-0.282038,...,5.537749,-0.180579,-0.380235,-1.436842,0.0,-0.235702,-0.180579,-0.209657,-0.415740,-0.180579
2,-0.282038,-0.011462,-0.483494,-0.235702,-0.235702,-1.308290,1.280191,-0.180579,0.532714,-0.282038,...,-0.180579,-0.180579,-0.380235,0.695971,0.0,-0.235702,-0.180579,-0.209657,-0.415740,-0.180579
3,-0.282038,0.549482,-0.483494,-0.235702,-0.235702,0.783805,-0.781133,-0.180579,0.532714,-0.282038,...,-0.180579,-0.180579,-0.380235,0.695971,0.0,-0.235702,-0.180579,-0.209657,-0.415740,-0.180579
4,-0.282038,1.011435,2.068279,-0.235702,-0.235702,0.015281,-0.781133,-0.180579,0.532714,-0.282038,...,-0.180579,-0.180579,-0.380235,0.695971,0.0,-0.235702,-0.180579,-0.209657,-0.415740,-0.180579
